In [1]:
# Cell 1

import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.video.io.VideoFileClip import VideoFileClip
from scipy import misc

%matplotlib inline

In [2]:
# Cell 2

class Undistorter:
    def __init__(self):
        (mtx, dist) = Undistorter.calibrate_camera()
        self.mtx = mtx
        self.dist = dist

    @staticmethod
    def calibrate_camera():
        # size of the board: 10x7, but the code wants the corners
        width = 9
        height = 6

        images = glob.glob("camera_cal/calibration*.jpg")

        objpoints = [] # 3d points in real world space
        imgpoints = [] # 2d points in image space

        # Prepare obj points
        objp = np.zeros((width*height, 3), np.float32)
        objp[:,:2] = np.mgrid[0:width,0:height].T.reshape(-1,2) # x,y coordinates

        shape = None

        for fname in images:

            # Read in the image in RGB
            img = mpimg.imread(fname)

            # Convert to grayscale
            gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
            shape = gray.shape[::-1]

            # Find the chessboard corners
            ret, corners = cv2.findChessboardCorners(gray, (width, height), None)

            # If corners are found, collect them
            if ret == True:
                imgpoints.append(corners)
                objpoints.append(objp)

                # Draw and display the corners
                img = cv2.drawChessboardCorners(img, (width, height), corners, ret)

        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, shape, None, None)
        return (mtx, dist)

    def undistort_img(self, img):
        undist = cv2.undistort(img, self.mtx, self.dist, None, self.mtx)
        return undist

In [ ]:
# Cell 3

def example_image_undistort(path):
    img = mpimg.imread(path)
    undist = Undistorter().undistort_img(img)
    plt.figure(figsize=(12,9))
    plt.subplot(1, 2, 1)
    plt.title('Original Image')
    plt.imshow(img)
    plt.subplot(1, 2, 2)
    plt.title('Undistorted Image')
    plt.imshow(undist)
    
#example_image_undistort("camera_cal/calibration4.jpg")
example_image_undistort("test_images/test2.jpg")

In [3]:
# Cell 4

class Thresholder:
    
    def thresholded_binary_image(self, undistorted):
        sobel_kernel = 3
        # Convert to grayscale
        gray = cv2.cvtColor(undistorted, cv2.COLOR_RGB2GRAY)
        # Take both Sobel x and y gradients
        sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0, ksize=sobel_kernel)
        sobely = cv2.Sobel(gray, cv2.CV_64F, 0, 1, ksize=sobel_kernel)

        mag_binary = self.__mag_thresh(sobelx, sobely)
        dir_binary = self.__dir_threshold(sobelx, sobely)
        color_binary = self.__color_threshold(undistorted)

        combined = np.zeros_like(dir_binary)
        combined[((color_binary == 1) & ((mag_binary == 1) | (dir_binary == 1)))] = 1
        return color_binary
    
    def __mag_thresh(self, sobelx, sobely, mag_thresh=(50, 255)):
        # Calculate the gradient magnitude
        gradmag = np.sqrt(sobelx**2 + sobely**2)
        # Rescale to 8 bit
        scale_factor = np.max(gradmag)/255 
        gradmag = (gradmag/scale_factor).astype(np.uint8) 
        # Create a binary image of ones where threshold is met, zeros otherwise
        binary_output = np.zeros_like(gradmag)
        binary_output[(gradmag >= mag_thresh[0]) & (gradmag <= mag_thresh[1])] = 1
        return binary_output

    # Define a function to threshold an image for a given range and Sobel kernel
    def __dir_threshold(self, sobelx, sobely, thresh=(0.7, 1.2)):
        # Take the absolute value of the gradient direction, 
        # apply a threshold, and create a binary image result
        absgraddir = np.arctan2(np.absolute(sobely), np.absolute(sobelx))
        binary_output =  np.zeros_like(absgraddir)
        binary_output[(absgraddir >= thresh[0]) & (absgraddir <= thresh[1])] = 1
        return binary_output

    def __color_threshold(self, undistorted, s_thresh=(170, 255)):
        # Convert to HLS color space and separate the S channel
        # Note: img is the undistorted image
        hls = cv2.cvtColor(undistorted, cv2.COLOR_RGB2HLS)
        s_channel = hls[:,:,2]

        # Threshold color channel
        s_binary = np.zeros_like(s_channel)
        s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1

        return s_binary

In [ ]:
# Cell 5

def example_thresholded_binary_image():
    img = mpimg.imread("test_images/test2.jpg")
    undist = Undistorter().undistort_img(img)
    plt.figure(figsize=(12,9))
    plt.imshow(Thresholder().thresholded_binary_image(undist), cmap='gray')
    
example_thresholded_binary_image()

In [4]:
# Cell 6

class Warper:
    def __init__(self, img_height, img_width):
        self.img_size = (img_width, img_height)
        self.src = np.float32(
            [[(img_width / 2) - 55, img_height / 2 + 100],
            [((img_width / 6) - 10), img_height],
            [(img_width * 5 / 6) + 60, img_height],
            [(img_width / 2 + 55), img_height / 2 + 100]])
        self.dst = np.float32(
            [[(img_width / 4), 0],
            [(img_width / 4), img_height],
            [(img_width * 3 / 4), img_height],
            [(img_width * 3 / 4), 0]])
        self.M = cv2.getPerspectiveTransform(self.src, self.dst)
        self.Minv = cv2.getPerspectiveTransform(self.dst, self.src)

    def warp(self, img):
        return cv2.warpPerspective(img, self.M, self.img_size, flags=cv2.INTER_LINEAR)
    
    def unwarp(self, img):
        return cv2.warpPerspective(img, self.Minv, self.img_size, flags=cv2.INTER_LINEAR)

In [ ]:
# Cell 7

def example_warped_image():
    img = mpimg.imread("test_images/straight_lines2.jpg")
    undist = Undistorter().undistort_img(img)
    warper = Warper(img.shape[0], img.shape[1])
    warped = warper.warp(undist)
    
    src = warper.src
    dst = warper.dst
    
    pts = dst.astype(np.int32)
    pts = pts.reshape((-1,1,2))
    cv2.polylines(warped,[pts],True,(255,0,0),3)
    
    pts = src.astype(np.int32)
    pts = pts.reshape((-1,1,2))
    cv2.polylines(undist,[pts],True,(255,0,0),3)
    
    plt.figure(figsize=(12,9))
    plt.subplot(1, 2, 1)
    plt.title('Undistorted Image with source points drawn')
    plt.imshow(undist)
    plt.subplot(1, 2, 2)
    plt.title('Warped result with dest. points drawn')
    plt.imshow(warped)
    
example_warped_image()

In [6]:
# Cell 8

class Polyfit:
    
    def polyfit(self, binary_warped):
        # Assuming you have created a warped binary image called "binary_warped"
        # Take a histogram of the bottom half of the image
        histogram = np.sum(binary_warped[int(binary_warped.shape[0]/2):,:], axis=0)
        # Create an output image to draw on and  visualize the result
        out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
        # Find the peak of the left and right halves of the histogram
        # These will be the starting point for the left and right lines
        midpoint = np.int(histogram.shape[0]/2)
        leftx_base = np.argmax(histogram[:midpoint])
        rightx_base = np.argmax(histogram[midpoint:]) + midpoint

        # Choose the number of sliding windows
        nwindows = 9
        # Set height of windows
        window_height = np.int(binary_warped.shape[0]/nwindows)
        # Identify the x and y positions of all nonzero pixels in the image
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Current positions to be updated for each window
        leftx_current = leftx_base
        rightx_current = rightx_base
        # Set the width of the windows +/- margin
        margin = 100
        # Set minimum number of pixels found to recenter window
        minpix = 50
        # Create empty lists to receive left and right lane pixel indices
        left_lane_inds = []
        right_lane_inds = []

        # Step through the windows one by one
        for window in range(nwindows):
            # Identify window boundaries in x and y (and right and left)
            win_y_low = binary_warped.shape[0] - (window+1)*window_height
            win_y_high = binary_warped.shape[0] - window*window_height
            win_xleft_low = leftx_current - margin
            win_xleft_high = leftx_current + margin
            win_xright_low = rightx_current - margin
            win_xright_high = rightx_current + margin
            # Draw the windows on the visualization image
            cv2.rectangle(out_img,(win_xleft_low,win_y_low),(win_xleft_high,win_y_high),(0,255,0), 2) 
            cv2.rectangle(out_img,(win_xright_low,win_y_low),(win_xright_high,win_y_high),(0,255,0), 2) 
            # Identify the nonzero pixels in x and y within the window
            good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
            good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
            # Append these indices to the lists
            left_lane_inds.append(good_left_inds)
            right_lane_inds.append(good_right_inds)
            # If you found > minpix pixels, recenter next window on their mean position
            if len(good_left_inds) > minpix:
                leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
            if len(good_right_inds) > minpix:        
                rightx_current = np.int(np.mean(nonzerox[good_right_inds]))

        # Concatenate the arrays of indices
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)

        # Extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds] 

        # Fit a second order polynomial to each
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)
        
        return (left_fit, right_fit)
    
    def non_sliding_window(self, binary_warped):
        # Assume you now have a new warped binary image 
        # from the next frame of video (also called "binary_warped")
        # It's now much easier to find line pixels!
        nonzero = binary_warped.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        margin = 100
        left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 
        right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

        # Again, extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]
        # Fit a second order polynomial to each
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2) 
        
#     def measure_curvature(self):
#         # Define conversions in x and y from pixels space to meters
#         ym_per_pix = 30/720 # meters per pixel in y dimension
#         xm_per_pix = 3.7/700 # meters per pixel in x dimension

#         # Fit new polynomials to x,y in world space
#         left_fit_cr = np.polyfit(ploty*ym_per_pix, leftx*xm_per_pix, 2)
#         right_fit_cr = np.polyfit(ploty*ym_per_pix, rightx*xm_per_pix, 2)
#         # Calculate the new radii of curvature
#         left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
#         right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
#         # Now our radius of curvature is in meters
#         print(left_curverad, 'm', right_curverad, 'm')
#         # Example values: 632.1 m    626.2 m

class PolyfitDrawer:
    def draw(self, warper, undist, warped, left_fit, right_fit):
        # Generate x and y values for plotting
        ploty = np.linspace(0, warped.shape[0]-1, warped.shape[0] )
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2] 
        
        # Create an image to draw the lines on
        warp_zero = np.zeros_like(warped).astype(np.uint8)
        color_warp = np.dstack((warp_zero, warp_zero, warp_zero))

        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        pts = np.hstack((pts_left, pts_right))

        # Draw the lane onto the warped blank image
        cv2.fillPoly(color_warp, np.int_([pts]), (0,255, 0))

        # Warp the blank back to original image space using inverse perspective matrix (Minv)
        newwarp = warper.unwarp(color_warp)
        # Combine the result with the original image
        result = cv2.addWeighted(undist, 1, newwarp, 0.3, 0)
        return result

In [ ]:
# Cell 9

def example_polyfit_image():
    img = mpimg.imread("test_images/test2.jpg")
    undist = Undistorter().undistort_img(img)
    binary = Thresholder().thresholded_binary_image(undist);
    warper = Warper(img.shape[0], img.shape[1])
    binary_warped = warper.warp(binary) 
    
    (left_fit, right_fit) = Polyfit().polyfit(binary_warped)
    
    fit_lines = PolyfitDrawer().draw(warper, undist, binary_warped, left_fit, right_fit)
    
    plt.figure(figsize=(12,9))
    plt.imshow(fit_lines)

example_polyfit_image()

In [8]:
# Cell 10

class Pipeline:
    def __init__(self, video_path):
        self.video_path = video_path
        self.video = VideoFileClip('{}.mp4'.format(video_path))
        self.undistorter = Undistorter()
        self.thresholder = Thresholder()
        self.warper = Warper(self.video.size[1], self.video.size[0])
        self.polyfitter = Polyfit()
        self.polyfit_drawer = PolyfitDrawer()

    def process_video(self):
        output = '{}_result.mp4'.format(self.video_path)
        clip1 = self.video.subclip(30, 51)
        output_clip = clip1.fl_image(self.__process_image)
        output_clip.write_videofile(output, audio=False)
        
    def __process_image(self, img):
        undist = self.undistorter.undistort_img(img)
        binary = self.thresholder.thresholded_binary_image(undist);
        binary_warped = self.warper.warp(binary) 

        (left_fit, right_fit) = self.polyfitter.polyfit(binary_warped)

        fit_lines = self.polyfit_drawer.draw(self.warper, undist, binary_warped, left_fit, right_fit)
        return fit_lines 
        
# pipeline = Pipeline('harder_challenge_video')
# pipeline = Pipeline('challenge_video')
pipeline = Pipeline('project_video')
pipeline.process_video()

[MoviePy] >>>> Building video project_video_result.mp4
[MoviePy] Writing video project_video_result.mp4


100%|█████████▉| 525/526 [01:52<00:00,  4.89it/s]


[MoviePy] Done.
[MoviePy] >>>> Video ready: project_video_result.mp4 



In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# import glob
# import cv2

# # Read in a thresholded image
# warped = mpimg.imread('warped_example.jpg')
# # window settings
# window_width = 50 
# window_height = 80 # Break image into 9 vertical layers since image height is 720
# margin = 100 # How much to slide left and right for searching

# def window_mask(width, height, img_ref, center,level):
#     output = np.zeros_like(img_ref)
#     output[int(img_ref.shape[0]-(level+1)*height):int(img_ref.shape[0]-level*height),max(0,int(center-width/2)):min(int(center+width/2),img_ref.shape[1])] = 1
#     return output

# def find_window_centroids(image, window_width, window_height, margin):
    
#     window_centroids = [] # Store the (left,right) window centroid positions per level
#     window = np.ones(window_width) # Create our window template that we will use for convolutions
    
#     # First find the two starting positions for the left and right lane by using np.sum to get the vertical image slice
#     # and then np.convolve the vertical image slice with the window template 
    
#     # Sum quarter bottom of image to get slice, could use a different ratio
#     l_sum = np.sum(warped[int(3*warped.shape[0]/4):,:int(warped.shape[1]/2)], axis=0)
#     l_center = np.argmax(np.convolve(window,l_sum))-window_width/2
#     r_sum = np.sum(warped[int(3*warped.shape[0]/4):,int(warped.shape[1]/2):], axis=0)
#     r_center = np.argmax(np.convolve(window,r_sum))-window_width/2+int(warped.shape[1]/2)
    
#     # Add what we found for the first layer
#     window_centroids.append((l_center,r_center))
    
#     # Go through each layer looking for max pixel locations
#     for level in range(1,(int)(warped.shape[0]/window_height)):
# 	    # convolve the window into the vertical slice of the image
# 	    image_layer = np.sum(warped[int(warped.shape[0]-(level+1)*window_height):int(warped.shape[0]-level*window_height),:], axis=0)
# 	    conv_signal = np.convolve(window, image_layer)
# 	    # Find the best left centroid by using past left center as a reference
# 	    # Use window_width/2 as offset because convolution signal reference is at right side of window, not center of window
# 	    offset = window_width/2
# 	    l_min_index = int(max(l_center+offset-margin,0))
# 	    l_max_index = int(min(l_center+offset+margin,warped.shape[1]))
# 	    l_center = np.argmax(conv_signal[l_min_index:l_max_index])+l_min_index-offset
# 	    # Find the best right centroid by using past right center as a reference
# 	    r_min_index = int(max(r_center+offset-margin,0))
# 	    r_max_index = int(min(r_center+offset+margin,warped.shape[1]))
# 	    r_center = np.argmax(conv_signal[r_min_index:r_max_index])+r_min_index-offset
# 	    # Add what we found for that layer
# 	    window_centroids.append((l_center,r_center))

#     return window_centroids

# window_centroids = find_window_centroids(warped, window_width, window_height, margin)

# # If we found any window centers
# if len(window_centroids) > 0:

#     # Points used to draw all the left and right windows
#     l_points = np.zeros_like(warped)
#     r_points = np.zeros_like(warped)

#     # Go through each level and draw the windows 	
#     for level in range(0,len(window_centroids)):
#         # Window_mask is a function to draw window areas
# 	    l_mask = window_mask(window_width,window_height,warped,window_centroids[level][0],level)
# 	    r_mask = window_mask(window_width,window_height,warped,window_centroids[level][1],level)
# 	    # Add graphic points from window mask here to total pixels found 
# 	    l_points[(l_points == 255) | ((l_mask == 1) ) ] = 255
# 	    r_points[(r_points == 255) | ((r_mask == 1) ) ] = 255

#     # Draw the results
#     template = np.array(r_points+l_points,np.uint8) # add both left and right window pixels together
#     zero_channel = np.zeros_like(template) # create a zero color channel
#     template = np.array(cv2.merge((zero_channel,template,zero_channel)),np.uint8) # make window pixels green
#     warpage = np.array(cv2.merge((warped,warped,warped)),np.uint8) # making the original road pixels 3 color channels
#     output = cv2.addWeighted(warpage, 1, template, 0.5, 0.0) # overlay the orignal road image with window results
 
# # If no window centers found, just display orginal road image
# else:
#     output = np.array(cv2.merge((warped,warped,warped)),np.uint8)

# # Display the final results
# plt.imshow(output)
# plt.title('window fitting results')
# plt.show()

In [ ]:
# import numpy as np
# import matplotlib.pyplot as plt
# # Generate some fake data to represent lane-line pixels
# ploty = np.linspace(0, 719, num=720)# to cover same y-range as image
# quadratic_coeff = 3e-4 # arbitrary quadratic coefficient
# # For each y position generate random x position within +/-50 pix
# # of the line base position in each case (x=200 for left, and x=900 for right)
# leftx = np.array([200 + (y**2)*quadratic_coeff + np.random.randint(-50, high=51) 
#                               for y in ploty])
# rightx = np.array([900 + (y**2)*quadratic_coeff + np.random.randint(-50, high=51) 
#                                 for y in ploty])

# leftx = leftx[::-1]  # Reverse to match top-to-bottom in y
# rightx = rightx[::-1]  # Reverse to match top-to-bottom in y


# # Fit a second order polynomial to pixel positions in each fake lane line
# left_fit = np.polyfit(ploty, leftx, 2)
# left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
# right_fit = np.polyfit(ploty, rightx, 2)
# right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]

# # Plot up the fake data
# mark_size = 3
# plt.plot(leftx, ploty, 'o', color='red', markersize=mark_size)
# plt.plot(rightx, ploty, 'o', color='blue', markersize=mark_size)
# plt.xlim(0, 1280)
# plt.ylim(0, 720)
# plt.plot(left_fitx, ploty, color='green', linewidth=3)
# plt.plot(right_fitx, ploty, color='green', linewidth=3)
# plt.gca().invert_yaxis() # to visualize as we do the images

